In [45]:
import json
import os
import ipdb

In [40]:
def parsePcap(filename, output):
    outputPath = os.path.join(output, "{}.json".format(filename.split('/')[-1]))
    cmd = 'tshark -r {} -Y "ssl" -T json -e frame.time -e ssl.app_data -e ip.src -e tcp.dstport -e tcp.srcport >> {}'.format(filename, outputPath)
    print(cmd)
    os.system(cmd)
parsePcap('../../../https-order/https_sample/file_explore.pcap', '../../../https-order/https_sample/')
parsePcap('../../../https-order/https_sample/shell_whoami.pcap', '../../../https-order/https_sample/')
parsePcap('../../../https-order/https_sample/sleep_10.pcap', '../../../https-order/https_sample/')
parsePcap('../../../https-order/https_sample/hashdump.pcap', '../../../https-order/https_sample/')
parsePcap('../../../https-order/https_sample/screenshot.pcap', '../../../https-order/https_sample/')
parsePcap('../../../https-order/https_sample/心跳包.pcap', '../../../https-order/https_sample/')
parsePcap('../../../https-order/https_only.pcap', '../../../https-order/')

tshark -r ../../../https-order/https_sample/file_explore.pcap -Y "ssl" -T json -e frame.time -e ssl.app_data -e ip.src -e tcp.dstport -e tcp.srcport >> ../../../https-order/https_sample/file_explore.pcap.json
tshark -r ../../../https-order/https_sample/shell_whoami.pcap -Y "ssl" -T json -e frame.time -e ssl.app_data -e ip.src -e tcp.dstport -e tcp.srcport >> ../../../https-order/https_sample/shell_whoami.pcap.json
tshark -r ../../../https-order/https_sample/sleep_10.pcap -Y "ssl" -T json -e frame.time -e ssl.app_data -e ip.src -e tcp.dstport -e tcp.srcport >> ../../../https-order/https_sample/sleep_10.pcap.json
tshark -r ../../../https-order/https_sample/hashdump.pcap -Y "ssl" -T json -e frame.time -e ssl.app_data -e ip.src -e tcp.dstport -e tcp.srcport >> ../../../https-order/https_sample/hashdump.pcap.json
tshark -r ../../../https-order/https_sample/screenshot.pcap -Y "ssl" -T json -e frame.time -e ssl.app_data -e ip.src -e tcp.dstport -e tcp.srcport >> ../../../https-order/https_sam

In [41]:
# 处理所有分完流后的https数据为json数据
dataPath = "../../../https-order/https-flows/"
filenames = [os.path.join(dataPath, x) for x in os.listdir(dataPath)]
for filename in filenames:
    parsePcap(filename, '../../../https-order/https-json/')

tshark -r ../../../https-order/https-flows/https_only-0114.pcap -Y "ssl" -T json -e frame.time -e ssl.app_data -e ip.src -e tcp.dstport -e tcp.srcport >> ../../../https-order/https-json/https_only-0114.pcap.json
tshark -r ../../../https-order/https-flows/https_only-0048.pcap -Y "ssl" -T json -e frame.time -e ssl.app_data -e ip.src -e tcp.dstport -e tcp.srcport >> ../../../https-order/https-json/https_only-0048.pcap.json
tshark -r ../../../https-order/https-flows/https_only-0115.pcap -Y "ssl" -T json -e frame.time -e ssl.app_data -e ip.src -e tcp.dstport -e tcp.srcport >> ../../../https-order/https-json/https_only-0115.pcap.json
tshark -r ../../../https-order/https-flows/https_only-0347.pcap -Y "ssl" -T json -e frame.time -e ssl.app_data -e ip.src -e tcp.dstport -e tcp.srcport >> ../../../https-order/https-json/https_only-0347.pcap.json
tshark -r ../../../https-order/https-flows/https_only-0053.pcap -Y "ssl" -T json -e frame.time -e ssl.app_data -e ip.src -e tcp.dstport -e tcp.srcport >

In [73]:
# 处理json数据，从中得出ip， app_len
def parseJson(filename):
    with open(filename, 'r') as f:
        data = json.loads(f.read())
    data = [x['_source']['layers'] for x in data]
    result = {}
    lens = []
    for item in data:
        ip = item['ip.src'][0]
        if ip.startswith('192.168'):
            label = "client"
        else:
            label = "server"
#         ipdb.set_trace()
        ssl_data = item.get("ssl.app_data", [""])
        l = len(ssl_data[0])
        lens.append((label, l))
    if len(data) == 0:
        return None
    clientSum = 0
    serverSum = 0
    for lenItem in lens:
        if lenItem[0] == 'client':
            clientSum += lenItem[1]
        else:
            serverSum += lenItem[1]
    result['start_time'] = data[0]['frame.time'][0]
    result['end_time'] = data[-1]['frame.time'][0]
    dstport = data[0]['tcp.dstport'][0]
    srcport = data[0]['tcp.srcport'][0]
    client_port = None
    if dstport in ['443', '80']:
        client_port = srcport
    else:
        client_port = dstport
    result['client_port'] = client_port
    result['clientSum'] = clientSum
    result['serverSum'] = serverSum
    result['lens'] = lens
    return result
parseJson('../../../https-order/https-json/https_only-0118.pcap.json')

{'start_time': 'Sep 28, 2021 15:57:42.831949000 CST',
 'end_time': 'Sep 28, 2021 15:57:43.504649000 CST',
 'client_port': '54715',
 'clientSum': 1775,
 'serverSum': 18190,
 'lens': [('client', 0),
  ('server', 0),
  ('server', 0),
  ('client', 0),
  ('server', 0),
  ('client', 1775),
  ('server', 0),
  ('server', 5759),
  ('server', 12431)]}

In [75]:
# 打印出样本所有数据

def printResult(result, file=None):
    print(result['start_time'])
    print(result['end_time'])
    print(result['client_port'])
    print("clientSum:", result['clientSum'])
    print("serverSum:", result['serverSum'])
    lens = result['lens']
    for item in lens:
        print("\t {}".format(item))
    print()
    
    if file:
        f = open(file, 'a+')
        print(result['start_time'], file=f)
        print(result['end_time'], file=f)
        print(result['client_port'], file=f)
        print("clientSum:", result['clientSum'], file=f)
        print("serverSum:", result['serverSum'], file=f)
        lens = result['lens']
        for item in lens:
            print("\t {}".format(item), file=f)
        print(file=f)
        f.close()

samplePath = "../../../https-order/https_sample/"
sampleFiles = [os.path.join(samplePath, x) for x in os.listdir(samplePath) if 'json' in x]
for file in sampleFiles:
    print(file.split('/')[-1])
    result = parseJson(file)
    printResult(result)

hashdump.pcap.json
Sep  3, 2021 15:39:50.507111000 CST
Sep  3, 2021 15:40:36.217426000 CST
51875
clientSum: 12857
serverSum: 458241
	 ('server', 0)
	 ('client', 0)
	 ('server', 0)
	 ('client', 1775)
	 ('server', 0)
	 ('server', 18095)
	 ('client', 0)
	 ('server', 0)
	 ('server', 0)
	 ('client', 0)
	 ('server', 0)
	 ('client', 1775)
	 ('server', 0)
	 ('server', 18095)
	 ('client', 0)
	 ('server', 0)
	 ('server', 0)
	 ('client', 0)
	 ('server', 0)
	 ('client', 1775)
	 ('server', 0)
	 ('server', 18095)
	 ('client', 0)
	 ('server', 0)
	 ('server', 0)
	 ('client', 0)
	 ('server', 0)
	 ('client', 1775)
	 ('server', 0)
	 ('server', 18095)
	 ('client', 0)
	 ('server', 0)
	 ('server', 0)
	 ('client', 0)
	 ('server', 0)
	 ('client', 1775)
	 ('server', 1487)
	 ('server', 49343)
	 ('server', 48575)
	 ('server', 49343)
	 ('server', 49343)
	 ('server', 49343)
	 ('server', 49343)
	 ('server', 49343)
	 ('server', 3551)
	 ('client', 0)
	 ('server', 0)
	 ('server', 0)
	 ('client', 0)
	 ('server', 0)
	 (

In [76]:
jsonPath = '../../../https-order/https-json/'
jsonFiles = [os.path.join(jsonPath, x) for x in os.listdir(jsonPath)]
httpsAllResult = []
for file in jsonFiles:
    result = parseJson(file)
    if result:
        httpsAllResult.append(result)
httpsAllResult = sorted(httpsAllResult, key=lambda x: x['start_time'])

In [77]:
for item in httpsAllResult:
    printResult(item, './httpsAllResult.txt')

Sep 28, 2021 15:51:46.845272000 CST
Sep 28, 2021 15:51:47.772421000 CST
54599
clientSum: 1775
serverSum: 18047
	 ('client', 0)
	 ('server', 0)
	 ('server', 0)
	 ('server', 0)
	 ('client', 0)
	 ('server', 0)
	 ('client', 1775)
	 ('server', 0)
	 ('server', 18047)

Sep 28, 2021 15:51:52.944036000 CST
Sep 28, 2021 15:51:53.653908000 CST
54600
clientSum: 1775
serverSum: 18190
	 ('client', 0)
	 ('server', 0)
	 ('server', 0)
	 ('server', 0)
	 ('client', 0)
	 ('server', 0)
	 ('client', 1775)
	 ('server', 0)
	 ('server', 5759)
	 ('server', 12431)

Sep 28, 2021 15:51:58.833815000 CST
Sep 28, 2021 15:51:59.754569000 CST
54601
clientSum: 1775
serverSum: 18430
	 ('client', 0)
	 ('server', 0)
	 ('server', 0)
	 ('client', 0)
	 ('server', 0)
	 ('client', 1775)
	 ('server', 1439)
	 ('server', 16991)

Sep 28, 2021 15:52:01.924255000 CST
Sep 28, 2021 15:52:02.621535000 CST
54602
clientSum: 1775
serverSum: 18238
	 ('client', 0)
	 ('server', 0)
	 ('server', 0)
	 ('server', 0)
	 ('client', 0)
	 ('server', 0